In [1]:
import os

os.chdir("../..")

In [2]:
%load_ext autoreload
%autoreload 2

from mars.models_training import data_loading, datasets
# from mars.models_training.notebook_helpers import evaluate

import pandas as pd
import os
from sklearn.model_selection import train_test_split
import numpy as np
from mars.models_training import document_level_similarity_search

2021-10-21 17:25:22.140027: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-21 17:25:22.140067: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-10-21 17:25:25.131480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-21 17:25:25.131544: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-21 17:25:25.131564: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-8E43FDR): /proc/driver/nvidia/version does not exist
2021-10-21 17:25:25.131817: I tensorflow/core/platform/cpu_fe

In [3]:
X, y = data_loading.load_document_level_issues_dataset(datasets.DocumentLevelDataset.ethics_ai_ethics, "laser")

0it [00:00, ?it/s]


In [10]:
X

array([], dtype=float64)

In [7]:
datasets.targets[datasets.DocumentLevelDataset.jobin2019]

['Transparency',
 'Justice, fairness, and equity',
 'Non-maleficence',
 'Responsibility and accountability',
 'Privacy',
 'Beneficence',
 'Freedom and autonomy',
 'Trust',
 'Sustainability',
 'Dignity',
 'Solidarity']

In [8]:
datasets.labels[datasets.DocumentLevelDataset.jobin2019]

,Transparency,"Justice, fairness, and equity",Non-maleficence,Responsibility and accountability,Privacy,Beneficence,Freedom and autonomy,Trust,Sustainability,Dignity,Solidarity
link,,,,,,,,,,,
https://www.semanticscholar.org/paper/a4d513cfc9d4902ef1a80198582f29b8ba46ac28,0,0,1,0,0,0,0,0,0,0,0
https://www.telekom.com/en/company/digital-responsibility/details/artificial-intelligence-ai-guideline-524366,1,1,1,1,1,1,0,1,0,0,0
https://www.ibm.com/blogs/think/2017/01/ibm-cognitive-principles/,1,0,0,0,0,0,0,1,0,0,0
https://www.gov.uk/government/publications/code-of-conduct-for-data-driven-health-and-care-technology/initial-code-of-conduct-for-data-driven-health-and-care-technology,1,1,1,0,1,0,0,0,0,0,0
http://www.fatml.org/resources/principles-for-accountable-algorithms,1,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
https://www.semanticscholar.org/paper/48eb44ee257b870de06a65d6f292ef6c7ad2e867,0,0,1,0,0,0,0,0,0,1,0
https://blog.openai.com/openai-charter/,0,0,1,0,0,1,0,0,0,0,0
https://www.agid.gov.it/it/agenzia/stampa-e-comunicazione/notizie/2018/03/21/lintelligenza-artificiale-al-servizio-del-cittadino-sfide-opportunita,0,0,0,1,0,0,0,0,0,0,0


In [10]:
datasets.labels[datasets.DocumentLevelDataset.ethics_ai_ethics].T.to_csv("jobin.csv")

In [8]:
df_labels = pd.read_csv('data/labels_hagendorffEthicsAIEthics2020.csv', index_col=0)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Baseline

In [11]:
X_max = list(map(max, X_test))

In [14]:
len(X_max)

83

In [18]:
X.shape

(330,)

In [12]:
evaluate(X_max, y_test)

NameError: name 'evaluate' is not defined

In [ ]:
def bucketize(scores, bins):
    return np.stack([np.histogram(x, range=(-1, 1), bins=bins)[0] for x in scores])

In [ ]:
from sklearn.feature_selection import RFE,RFECV

In [ ]:
buckets_num = 250

model = document_level_similarity_search.model
model.fit(X_train, y_train)

In [ ]:
model = document_level_similarity_search.model
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_train)[:, 1]
evaluate(y_pred, y_train)

In [ ]:
model2 = document_level_similarity_search.model_with_normalization
model2.fit(X_train, y_train)
y_pred = model2.predict_proba(X_train)[:, 1]
evaluate(y_pred, y_train)

In [ ]:
buckets_num = 250

model3 =  document_level_similarity_search.model_with_normalization_rfe
model3.set_params(bucketize__kw_args = {'bins': buckets_num})
model3.fit(X_train, y_train)
y_pred = model3.predict_proba(X_train)[:, 1]
evaluate(y_pred, y_train)

In [ ]:
param_grid = {
    "rfe__n_features_to_select": list(range(10,101,5)),
    "bucketize__kw_args":[{"bins":int(n)} for n in np.linspace(50,400,num=40)]
}

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


cv = RandomizedSearchCV(model3, param_grid, scoring="roc_auc",n_jobs=-1, verbose=1, n_iter=150)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
best_model = cv.best_estimator_
cv.best_score_, cv.best_params_

In [ ]:
y_pred = best_model.predict_proba(X_train)[:, 1]
evaluate(y_pred, y_train)

In [ ]:
y_pred = best_model.predict_proba(X_test)[:, 1]
evaluate(y_pred, y_test)

In [ ]:
import pickle

pickle.dump(best_model, open("models/ethics_ai_ethics_laser_model.pkl", "wb"))

In [ ]:
import pickle

model = pickle.load(open('models/ethics_ai_ethics_laser_model.pkl', 'rb'))

In [ ]:
model